In [1]:
search_for = 139
start_from = 0
threads = 10
thread = 2
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.08965349197387695
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 5
all cases: 376006121
type: [1, 1, 1, 1, 139] 139
cases of this type: 373301041
100000 10.997638230214752
200000 18.82483856234459
300000 18.96972471312677
400000 20.366902017596484
500000 19.435686263166886
600000 19.776029104964408
700000 19.066407331583314
800000 19.034516361694088
900000 20.697574193479976
1000000 20.951460049990562
1100000 21.514248057783405
1200000 18.204890982273824
1300000 21.959850516566775
1400000 5.2034113227045875
1500000 22.03096053729793
1600000 25.56442049343103
1700000 19.024317006569795
1800000 17.625711355182393
1900000 22.91284777447125
2000000 20.27901620795198
2100000 20.498576984133113
2200000 21.55989437244769
2300000 23.865737330831784
2400000 23.15170256655517
2500000 30.728739302661367
2600000 33.59569068937806
2700000 18.05778166558668
2800000 25.69301300822984
2900000 47.02110978020824
3000000 44.5

29700000 38.86612662924614
29800000 13.781664743279663
29900000 36.02988633730701
30000000 36.407895122366305
30100000 38.19946658004803
30200000 35.71403510873924
30300000 38.343593484054146
30400000 35.84651912141378
30500000 38.3536230327852
30600000 34.8139397430234
30700000 38.894893633865046
30800000 32.5140533100952
30900000 16.941544485472615
31000000 27.614275489483337
31100000 35.30602191314009
31200000 35.319669944635606
31300000 38.08580663364226
31400000 36.50961057012204
31500000 36.483562878766676
31600000 35.28646285288949
31700000 38.65934430119755
31800000 36.7764853166466
31900000 37.40648184278918
32000000 25.48915113352372
32100000 23.341021972281936
32200000 38.22224362444516
32300000 38.83009750172108
32400000 37.964260996892435
32500000 15.411945917163234
32600000 33.56943553982691
32700000 36.63495266604812
32800000 36.85841631868301
32900000 36.186119540877634
33000000 36.85747715071231
33100000 34.4201811317907
33200000 37.22064830065336
33300000 34.232532770

59700000 32.611088329797475
59800000 31.556890873075925
59900000 31.194624078258098
60000000 30.171885938996635
60100000 31.062255647669883
60200000 29.37740832154008
60300000 29.383257649098034
60400000 24.460813010296427
60500000 12.409184111311596
60600000 29.04887462048708
60700000 29.27319421785621
60800000 29.653552626615674
60900000 29.47876943119852
61000000 31.727701983820467
61100000 30.403042807525118
61200000 32.01001405370036
61300000 29.349894311536108
61400000 13.851336208605776
61500000 31.817725309344652
61600000 33.039019513287414
61700000 31.235594441437627
61800000 33.39184500167131
61900000 33.161837323702045
62000000 32.583997723536335
62100000 33.06214561121116
62200000 28.40456078201972
62300000 15.647620372925733
62400000 31.86878422033679
62500000 31.53908334197519
62600000 31.111612693349688
62700000 33.155333603739194
62800000 29.760100579071235
62900000 29.266014582016325
63000000 29.087663289846567
63100000 20.843501245017205
63200000 16.37178146040338
633

89400000 23.56279850765719
89500000 26.557849330783704
89600000 25.852617288383907
89700000 26.798669015514125
89800000 25.913250364547363
89900000 26.46488626242469
90000000 8.93872548370691
90100000 24.94070739751995
90200000 26.22764842993359
90300000 26.225833895774077
90400000 25.792759681371237
90500000 25.30247396420489
90600000 26.46174297051148
90700000 10.52113415279338
90800000 26.44643288699846
90900000 27.671050686472036
91000000 28.57163932480971
91100000 28.8693925650594
91200000 28.304624505163375
91300000 30.166808438558625
91400000 31.14605463297455
91500000 28.853498692733428
91600000 28.01059240951768
91700000 28.99138949165945
91800000 27.475914518454882
91900000 28.09096244332982
92000000 12.834419018702894
92100000 21.56492956352369
92200000 25.839951395962533
92300000 25.865136551020694
92400000 25.14505317174628
92500000 25.47069147236745
92600000 26.68401106755771
92700000 4.717596626481843
92800000 27.547697210351657
92900000 25.81588929871122
93000000 24.935

118500000 25.69772641650912
118600000 25.216046546465606
118700000 25.41308588521626
118800000 23.18022567500226
118900000 23.717096197626788
119000000 19.86115342554599
119100000 10.789207310180348
119200000 22.38047916898022
119300000 22.786721305100265
119400000 23.18348002214104
119500000 16.738444915955675
119600000 13.880050372223032
119700000 22.88065917685993
119800000 22.858741499918537
119900000 22.972199061487302
120000000 14.675284758542725
120100000 17.695333768160673
120200000 23.269370115695125
120300000 22.775285499384037
120400000 24.216963378442557
120500000 24.180023252872605
120600000 25.131789792225497
120700000 25.7983666812066
120800000 25.45886468612009
120900000 25.598703362124567
121000000 26.298568507734235
121100000 25.63546102284898
121200000 26.18957362791406
121300000 25.758435774428147
121400000 27.120660306145904
121500000 23.88391704390788
121600000 23.320690994770764
121700000 21.52962481990256
121800000 9.79066934080187
121900000 23.34333640898078
12

...

233100000 13.131574933232288
233200000 13.0983018209055
233300000 13.044468783106199
233400000 13.785974043365451
233500000 14.024640882628246
233600000 14.001182518974545
233700000 13.848083736871336
233800000 14.387570030786453
233900000 13.697859389233011
234000000 14.277808553872196
234100000 13.04879674863614
234200000 12.779820110527128
234300000 12.599657431814535
234400000 12.32606925860615
234500000 12.984634031048103
234600000 12.415108875820716
234700000 4.726824248429647
234800000 12.665698952176907
234900000 12.455955292531108
235000000 7.016625190031054
235100000 9.11165507055366
235200000 12.663042652298296
235300000 12.308963543227865
235400000 4.2486222242133085
235500000 13.235073960481802
235600000 11.94339737335134
235700000 13.404545782749477
235800000 12.660122134041304
235900000 13.533926908501005
236000000 12.796819470042747
236100000 14.517814917718162
236200000 13.448716809449932
236300000 14.196181846690331
236400000 13.09317573998495
236500000 13.60242568413

261700000 10.532128788879348
261800000 11.303305779537046
261900000 1.8121656274229418
262000000 11.941861824729697
262100000 10.12762249313058
262200000 11.460552497543368
262300000 10.617605299637733
262400000 6.204808614793595
262500000 8.010095388117202
262600000 11.263811429031596
262700000 11.555539259977357
262800000 11.148248585005568
262900000 12.583467711279134
263000000 10.867366476397796
263100000 12.097801846086272
263200000 11.7147004795496
263300000 11.462101434353173
263400000 11.425299964731895
263500000 11.708181172240945
263600000 10.769515225701642
263700000 11.695396578104857
263800000 9.902003725560512
263900000 11.093581566854864
264000000 4.972684549378511
264100000 9.772283032547625
264200000 9.291362295749302
264300000 11.152467829202203
264400000 9.118217406472095
264500000 10.17846674945914
264600000 3.4863512830976835
264700000 11.059079379022485
264800000 9.55670797185042
264900000 10.689331483960599
265000000 9.42279757244574
265100000 6.260630927467253
2

290800000 8.380357677209773
290900000 7.878055569487328
291000000 7.607532377907699
291100000 7.676517942125727
291200000 7.753821042641236
291300000 7.701559335162222
291400000 4.424495870527997
291500000 6.124691839408374
291600000 7.539521376664376
291700000 7.891320057428076
291800000 7.275950768712646
291900000 7.938202612328675
292000000 7.77478621160797
292100000 8.243256515189287
292200000 2.4941053865514395
292300000 8.894399348069207
292400000 8.070478237895626
292500000 9.008004527156704
292600000 7.981837660956967
292700000 9.141909579886095
292800000 8.139049217663013
292900000 9.337217750914833
293000000 8.115764417250933
293100000 8.811101637144894
293200000 8.244348495067495
293300000 5.894349418972481
293400000 5.804535986807613
293500000 8.267938856289838
293600000 7.640600027948089
293700000 7.733093531134355
293800000 7.3034032325180585
293900000 7.882925223363791
294000000 7.422811547241727
294100000 3.2041731983798667
294200000 6.6511700482620295
294300000 7.52788

320000000 2.414133066532135
320100000 5.636725753123133
320200000 5.5953117027185435
320300000 5.746888770355856
320400000 5.248093946912414
320500000 5.636754014532333
320600000 5.1079425886123335
320700000 5.327518622932509
320800000 4.7932020938482705
320900000 4.674781109657276
321000000 1.8890335525148776
321100000 5.355972183402823
321200000 4.758534736296982
321300000 5.450617067856351
321400000 5.159829850808765
321500000 5.76280585599479
321600000 5.11791419558555
321700000 5.657766349785057
321800000 5.375539780728143
321900000 3.8083611880148216
322000000 3.9037370208828097
322100000 5.740635264050209
322200000 5.303755820905701
322300000 5.600512049809607
322400000 5.509664472634431
322500000 5.476272562517629
322600000 4.868303180287791
322700000 2.3653466997090065
322800000 5.751487968509644
322900000 5.3853366024222655
323000000 5.477978696120703
323100000 4.948566218726276
323200000 5.264685485425085
323300000 4.8579336134506725
323400000 4.960026163082076
323500000 4.6

349000000 0.6916575904879364
349100000 3.0398864852880476
349200000 2.5577959928658625
349300000 1.7594748605046169
349400000 1.9785701772870818
349500000 2.829272181397826
349600000 2.47272643409023
349700000 2.7756651102309666
349800000 2.6287977431700664
349900000 2.6114993574683547
350000000 2.457927598516077
350100000 2.5933185910740515
350200000 2.4845507080650515
350300000 2.5498243279583277
350400000 2.294323503522951
350500000 0.9236710134609382
350600000 2.103961490334527
350700000 2.5390087174691813
350800000 2.48046525060605
350900000 2.379312384093779
351000000 2.4707992845580784
351100000 2.462024771302342
351200000 2.594554798766931
351300000 2.463070721275993
351400000 2.5555344655845134
351500000 2.493895286427184
351600000 2.7765574088508105
351700000 0.5829482129615537
351800000 2.815159177736235
351900000 2.4214835582975716
352000000 0.8948150856267763
352100000 2.36071176079458
352200000 2.589425038507587
352300000 2.285949228007651
352400000 2.5748166806351307
352